In [1]:
%load_ext lab_black

Step: Load data

In [2]:
import pandas as pd
import os
from pyprojroot import here
import numpy as np
from skimpy import clean_columns
from data_cleaning.fun_hot_encode_limit import fun_hot_encode_limit
import numpy as np
import pandas as pd
import gower
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram
import matplotlib.pyplot as plt
from sklearn.metrics import (
    silhouette_score,
    calinski_harabasz_score,
    davies_bouldin_score,
)


path_data = here("./data")
os.chdir(path_data)

data_draft_kings_original = pd.read_csv("draft_kings_2021_2022.csv")
data_draft_kings = pd.read_parquet("data_draft_kings_analysis.parquet")
data_draft_kings_analysis = data_draft_kings.drop(columns=["week"])

In [3]:
import pulp
import random

# Generate fake player data
players = {
    f"Player{i}": {
        "position": random.choice(["QB", "RB", "WR", "TE", "DST"]),
        "salary": random.randint(3000, 9000),
        "points": random.uniform(0, 40),
    }
    for i in range(1, 101)
}

In [5]:
pd.DataFrame(players)

,Player1,Player2,Player3,Player4,Player5,Player6,Player7,Player8,Player9,Player10,...,Player91,Player92,Player93,Player94,Player95,Player96,Player97,Player98,Player99,Player100
position,WR,QB,QB,TE,RB,QB,QB,QB,QB,DST,...,WR,QB,RB,QB,QB,WR,RB,RB,TE,TE
salary,8541,6369,4088,6762,4715,6242,8655,6800,8184,6296,...,4214,6517,8383,4259,3619,3109,3030,7193,4915,5010
points,15.104506,4.253877,7.250161,18.548211,31.94972,11.406805,0.365365,28.142896,20.546849,33.03067,...,9.918665,28.360955,8.603833,33.692006,3.704472,2.609307,25.610785,13.748802,35.838109,9.535819


In [6]:
# Define problem
problem = pulp.LpProblem("NFL_Milly_Maker", pulp.LpMaximize)

# Define variables
player_vars = pulp.LpVariable.dicts(
    "player", players.keys(), lowBound=0, upBound=1, cat="Integer"
)

In [10]:
# Define constraints
position_limits = {"QB": 1, "RB": 2, "WR": 3, "TE": 1, "DST": 1}

In [11]:
for position, limit in position_limits.items():
    problem += (
        pulp.lpSum(
            player_vars[p] for p in players if players[p]["position"] == position
        )
        == limit
    )

In [14]:
problem += pulp.lpSum(player_vars[p] * players[p]["salary"] for p in players) <= 50000

In [16]:
problem += pulp.lpSum(player_vars) == 9

In [18]:
problem += pulp.lpSum(player_vars[p] * players[p]["points"] for p in players)

In [19]:
# Define objective function

# Solve problem
status = problem.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /anaconda/envs/azureml_py38/lib/python3.8/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/ec9bb86d7321461599e606ee9c7cc078-pulp.mps max timeMode elapsed branch printingOptions all solution /tmp/ec9bb86d7321461599e606ee9c7cc078-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 12 COLUMNS
At line 613 RHS
At line 621 BOUNDS
At line 722 ENDATA
Problem MODEL has 7 rows, 100 columns and 300 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Problem is infeasible - 0.00 seconds
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00



In [20]:
# Display results
print(f"Status: {pulp.LpStatus[status]}")
print(f"Objective: {pulp.value(problem.objective)}")
print("Players in optimal lineup:")
for player in players:
    if pulp.value(player_vars[player]) == 1:
        print(f"{player}: {players[player]}")

Status: Infeasible
Objective: 318.4251563386727
Players in optimal lineup:
Player10: {'position': 'DST', 'salary': 6296, 'points': 33.030670457210164}
Player27: {'position': 'QB', 'salary': 5696, 'points': 39.042281605536175}
Player31: {'position': 'TE', 'salary': 4582, 'points': 39.815364215996674}
Player48: {'position': 'RB', 'salary': 3747, 'points': 37.187410420496136}
Player58: {'position': 'WR', 'salary': 3831, 'points': 29.523712763596784}
Player66: {'position': 'WR', 'salary': 7984, 'points': 32.64545668496963}
Player76: {'position': 'QB', 'salary': 6027, 'points': 35.53298280978587}
Player77: {'position': 'DST', 'salary': 7136, 'points': 37.82626582276134}
Player80: {'position': 'WR', 'salary': 8249, 'points': 32.75787248270365}
Player83: {'position': 'RB', 'salary': 5945, 'points': 39.38509442794828}
